In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from zipfile import ZipFile
from sklearn.ensemble import RandomForestRegressor

In [2]:
with ZipFile('Boston-house-prices.zip', 'r') as f:
    f.extractall()

In [4]:
boston_df = pd.read_csv('boston.csv', index_col=0)

#data exploration
boston_df.head()
boston_df.info()
boston_df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 506 entries, 0.00632 to 0.04741
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ZN       506 non-null    float64
 1   INDUS    506 non-null    float64
 2   CHAS     506 non-null    int64  
 3   NOX      506 non-null    float64
 4   RM       506 non-null    float64
 5   AGE      506 non-null    float64
 6   DIS      506 non-null    float64
 7   RAD      506 non-null    int64  
 8   TAX      506 non-null    float64
 9   PTRATIO  506 non-null    float64
 10  B        506 non-null    float64
 11  LSTAT    506 non-null    float64
 12  MEDV     506 non-null    float64
dtypes: float64(11), int64(2)
memory usage: 55.3 KB


,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [ ]:
train_df.fillna(train_df.mode(), inplace=True)
test_df.fillna(test_df.mode(), inplace=True)

In [ ]:
sample_submission_df = pd.read_csv('sample_submission.csv', index_col=False)
sample_submission_df.head
sample_submission_df.info()

In [ ]:
# number of rows with missing data
train_df.isnull().sum()[train_df.isnull().sum() > 0]

In [ ]:
# identify the percentage of missing values
train_missing_perectage = train_df.isnull().sum()[train_df.isnull().sum() > 0]/len(train_df)*100
train_missing_perectage

In [ ]:
# fill missing data values with mode
train_df = train_df.apply(lambda x: x.fillna(x.mode()[0]), axis=0)
train_df.isnull().sum()

In [ ]:
test_missing_perectage = test_df.isnull().sum()[test_df.isnull().sum() > 0]/len(train_df)*100
test_missing_perectage

In [ ]:
test_df = test_df.apply(lambda x: x.fillna(x.mode()[0]), axis=0)
test_df.isnull().sum()

In [ ]:
# Check for duplicates in the train and test data
print(f"Number of duplicate rows in the train dataframe: {train_df.duplicated().sum()}")
print(f"Number of duplicate rows in the test dataframe: {test_df.duplicated().sum()}")

In [ ]:
train_cat_columns = train_df.select_dtypes(include=['object', 'category']).columns.tolist()
train_num_columns = train_df.select_dtypes(exclude=['object']).columns.tolist()

In [ ]:
# Plot the distribution of SalePrice
plt.figure(figsize=(4, 6))
sns.histplot(train_df['SalePrice'], kde=True)
plt.title("SalePrice Distribution")
plt.xlabel('SalePrice')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# numerical columns for the training dataframe
train_df_numerical = train_df[train_num_columns]

# Correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(train_df_numerical.corr(), cmap='coolwarm')
plt.title("Correlation Heatmap for Training Data")
plt.show()

In [ ]:
# Set the figure size
plt.figure(figsize=(25, 25))

# Number of columns for the grid of subplots
num_columns = len(train_df_numerical.columns)

# Create box plots for each numerical column
for i, column in enumerate(train_df_numerical.columns):
    plt.subplot(num_columns // 3 + 1, 3, i + 1)  
    sns.boxplot(x=train_df_numerical[column])
    plt.title(f"Box Plot for {column}", fontsize=12)

# Manually adjust the layout to add more spacing between subplots
plt.subplots_adjust(hspace=2.5, wspace=0.5)  # Adjust vertical (hspace) and horizontal (wspace) spacing

# Display the plots
plt.show()